# IMPRORT

In [ ]:
from typing import Any
from py3.k_means import PixelValueGenerator, CustomKMeans
import numpy as np
from py3.CNN_model import Cplx_CustomCNN_1D
from py3.Attention_Layer import CustomAttentionLayer
from py3.classification import CustomClassifierModel
import tensorflow as tf

import re
import numpy as np

In [ ]:
class Color:
    RED = '\033[91m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    END = '\033[0m'

In [ ]:

vectors = []

with open('1_segment_test.txt', 'r') as file:
    for line in file:
        values = re.findall(r'\[([^]]*)\]', line)
        for sublist in values:
            vector = [float(value) for value in sublist.split(',')]
            vectors.append(vector)

numpy_array = np.array(vectors)

print("Shape of the numpy...:", numpy_array.shape)
reshaped_array = numpy_array.reshape(62, 73, 10)
shuffled_arrays = []
for _ in range(100):
    np.random.shuffle(reshaped_array)
    shuffled_arrays.append(reshaped_array.copy())


# Create the data variable
data = np.array(shuffled_arrays)
print("data shape:", data.shape)

n_clusters=2
custom_kmeans = CustomKMeans(n_clusters=n_clusters)

clustered_data = []
#
for i in range(data.shape[0]):
    custom_kmeans.fit(data[i])
    clustered_data.append(custom_kmeans.get_cluster_centers())
     
clusters = np.array(clustered_data)
print("clusters shape:", clusters.shape)
# add random labels to the clusters (between 0 and 7)
labels = np.random.randint(0, 8, clusters.shape[0])
print("labels shape:", labels.shape) 

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

train_data, test_data, train_labels, test_labels = train_test_split(clusters, labels, test_size=0.2, random_state=42)
train_labels = to_categorical(train_labels, num_classes=8)
test_labels = to_categorical(test_labels, num_classes=8)
 
print(Color.GREEN + "train_data shape:", str(train_data.shape) + Color.END)
print(Color.GREEN + "test_data shape:", str(test_data.shape) + Color.END)

tf.config.run_functions_eagerly(True) # SAVE MY LIFE !!!!!


In [12]:
class Custom_Model(tf.keras.Model):
    
    def __init__(self):
        super(Custom_Model, self).__init__()
        self.enc = Cplx_CustomCNN_1D()
        self.attn = CustomAttentionLayer(units=64)
        self.classifier = CustomClassifierModel(num_classes=8)
    
    def call(self, inputs):
        
        intermediate = tf.map_fn(self.enc, inputs, dtype=tf.float32)
        attention_outputs = tf.map_fn(lambda x: self.attn(self.enc(x)), inputs, dtype=tf.float32)
        emb = [tf.multiply(alpha, tensor) for alpha, tensor in zip(attention_outputs, intermediate)]
        emb = tf.reduce_sum(emb, axis=1)
        
        return self.classifier(emb)
    
    def summary(self):
        self.enc.summary()
        self.attn.summary()
        #self.classifier.summary()
        
         
model = Custom_Model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
history = model.fit(train_data, train_labels, epochs=1, batch_size=32, validation_data=(test_data, test_labels))
 

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_18 (Conv1D)          (None, 71, 256)           7936      
                                                                 
 batch_normalization_24 (Ba  (None, 71, 256)           1024      
 tchNormalization)                                               
                                                                 
 dropout_18 (Dropout)        (None, 71, 256)           0         
                                                                 
 conv1d_19 (Conv1D)          (None, 69, 256)           196864    
                                                                 
 batch_normalization_25 (Ba  (None, 69, 256)           1024      
 tchNormalization)                                               
                                                                 
 dropout_19 (Dropout)        (None, 69, 256)          